Este trabalho usa a biblioteca do plot.ly com _offline plot_. Tendo o `plot.ly` instalado e atualizado, ele mostra o gráfico no Jupyter sem precisar de credenciais. Mais informações [aqui](https://plot.ly/python/offline/#generating-offline-graphs-within-jupyter-notebook)

In [1]:
from matplotlib import pyplot as plt
import plotly.graph_objs as go
from plotly import tools
import pandas as pd
import numpy as np
import plotly

Inicializa ambiente offline pro plot.ly

In [2]:
plotly.offline.init_notebook_mode(connected=True)

Ignora erros de SSL

In [3]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

Permite mais de um output no `Out` da célula

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Projeto Final - Introdução a Ciência dos Dados

## League of Legends Ranked Matches

### Índice
* [Introdução](#Introdução)
* [Análise Inicial dos Dados](#Análise-Inicial-dos-Dados)
    * [Campeões](#Análise---Campeões)
    * [Partidas](#Análise---Partidas)
    * [Jogadores](#Análise---Jogadores)
    * [Estatísticas de Jogadores](#Análise---Estatísticas-de-Jogadores)
    * [Estatísticas dos Times](#Análise---Estatísticas-dos-Times)
* [Limpeza dos Dados](#Limpeza-dos-Dados)
    * [Renomeando Colunas](#Renomeando-colunas)
    * [Concatenando bases iguais](#Concatenando-bases-iguais)
    * [Trocando "vitória" de tabela](#Trocando-"vitória"-de-tabela)
    * [Conferindo valores](#Conferindo-valores)
    * [Última Limpeza](#Última-Limpeza)
* [Análise Exploratória](#Análise-Exploratória)
    * [Explorando Valores dos Jogadores](#Explorando-Valores-dos-Jogadores)
    * [Explorando Valores dos Times](#Explorando-Valores-dos-Times)
* [Perguntas](#Perguntas)

## Introdução

Este relatório é uma análise de dados de uma base sobre o jogo _League of Legends_. A base [League of Legends Ranked Matches](https://www.kaggle.com/paololol/league-of-legends-ranked-matches), retirada do site _Kaggle_, contém informações sobre 180.000 partidas ranqueadas do jogo, situadas em 6 temporadas (_seasons_ 3 a 8), aproximadamente entre os anos 2014 e 2017. Uma partida ranqueada é um modo competitivo de jogo que categoriza os jogadores em ranques, baseados no nível de habilidade. 

A base é dividida em 7 grandes arquivos. Nosso objetivo inicial é checar como os dados estão dispostos nos arquivos e fazer qualquer alteração necessária. Depois, vamos limpar os dados que possuírem incoerências. O próximo passo será gerar uma análise exploratória e, por fim, responder algumas perguntas interessantes sobre partidas ranqueadas em _League of Legends_. 


## Análise Inicial dos Dados

Aqui vamos carregar os arquivos e verificar como os dados estão formatados.

Obs: Visto o tamanho dos arquivos (alguns com 56 colunas) e a escassez de recursos computacionais, ficou previamente definido quais atributos seriam mais relevantes para o  trabalho prático.

O download dos arquivos é feito pelo `read_csv` do pandas, mas caso ele falhe, deixo comentado uma linha com a opção para abrir os arquivos de uma pasta chamada `data`. Seguem links para download:

* Champs: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/champs.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/champs.csv)
* Matches: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/matches.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/matches.csv)
* Participants: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/participants.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/participants.csv)
* Stats1: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/stats1.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/stats1.csv)
* Stats2: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/stats2.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/stats2.csv)
* Teamstats: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/teamstats.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/teamstats.csv)

### Análise - Campeões

* `name` (inteiro): nome do campeão
* `id` (inteiro): identificador único

In [5]:
# df_champs = pd.read_csv('./data/champs.csv')
df_champs = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/champs.csv')
df_champs.head()

print('shape:', df_champs.shape)

,name,id
0,Jax,24
1,Sona,37
2,Tristana,18
3,Varus,110
4,Fiora,114


shape: (138, 2)


### Análise - Partidas

* `id` (inteiro): identificador único
* `queueid` (inteiro): identificador de qual fila ranqueada a partida estava
* `seasonid` (inteiro): identificador da temporada

In [6]:
# df_matches = pd.read_csv('./data/matches.csv', usecols=['id', 'queueid', 'seasonid'])
df_matches = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/matches.csv',
                        usecols=['id', 'queueid', 'seasonid'])
df_matches.head()

print('shape:', df_matches.shape)

,id,queueid,seasonid
0,10,420,8
1,11,420,8
2,12,420,8
3,13,420,8
4,14,420,8


shape: (184069, 3)


### Análise - Jogadores

* `id` (inteiro): identificador único
* `matchid` (inteiro): referencia a tabela de partidas
* `player` (inteiro): 
    * 1-5: time azul
    * 6-10: time vermelho
* `championid` (inteiro): referencia a tabela de campeões
* `role` (string): função que o jogador exerceu
    * `SOLO`
    * `NONE`
    * `DUO_CARRY` 
    * `DUO_SUPPORT`
* `position` (string): parte do mapa onde o jogador estava
    * `BOT` 
    * `JUNGLE` 
    * `TOP`
    * `MID`

In [7]:
# df_players = pd.read_csv('./data/participants.csv', usecols=['id', 'matchid', 'player', 'championid', 'role', 'position'])
df_players = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/participants.csv',
                        usecols=['id', 'matchid', 'player', 'championid', 'role', 'position'])
df_players.head()

print('shape:', df_players.shape)

,id,matchid,player,championid,role,position
0,9,10,1,19,NONE,JUNGLE
1,10,10,2,267,DUO_SUPPORT,BOT
2,11,10,3,119,DUO_CARRY,BOT
3,12,10,4,114,SOLO,TOP
4,13,10,5,112,SOLO,MID


shape: (1834520, 6)


### Análise - Estatísticas de Jogadores

* `id` (inteiro): identificador único
* `win` (inteiro): jogador ganhou a partida em que participa?
* `kills` (inteiro): abatimentos feitos pelo jogador na partida
* `deaths` (inteiro): mortes na partida
* `assists` (inteiro): assistências em abatimentos na artida
* `visionscore` (inteiro): placar de visão, uma estatística sobre a contribuição em visão que o jogador teve. Geralmente entre 5 e 150 pontos
    * funciona da seguinte forma: cada minuto de sentinela (item que garante visão sobre uma área finita do mapa) colocada gera 1 ponto ao jogador (existem vários modificadores dependendo do tipo de sentinela colocada que podem aumentar esse valor). Toda vez que o jogador retira uma sentinela inimiga, o tempo de vida restante dela é convertida em pontos (1 minuto:1 ponto). Há vários modificadores também que dão pontos bônus base quando usados.
* `totminionskilled` (inteiro): o _farm_ do jogador, tropas inimigas (_minions_) e neutras (monstros da selva) abatidas em troca de ouro e experiência (não conta campeões)

In [8]:
# df_stats1 = pd.read_csv('./data/stats1.csv', usecols=['id', 'win', 'kills', 'deaths', 'assists', 'visionscore', 
#                                                       'totminionskilled']) 
df_stats1 = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/stats1.csv',
                       usecols=['id', 'win', 'kills', 'deaths', 'assists', 'visionscore', 'totminionskilled'])
df_stats1.head()

print('shape:', df_stats1.shape)

,id,win,kills,deaths,assists,visionscore,totminionskilled
0,9,0,6,10,1,14,42
1,10,0,0,2,12,30,17
2,11,0,7,8,5,26,205
3,12,0,5,11,2,5,164
4,13,0,2,8,2,15,235


shape: (999999, 7)


In [9]:
# df_stats2 = pd.read_csv('./data/stats2.csv', usecols=['id', 'win', 'kills', 'deaths', 'assists', 'visionscore', 
#                                                       'totminionskilled']) 
df_stats2 = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/stats2.csv',
                       usecols=['id', 'win', 'kills', 'deaths', 'assists', 'visionscore', 'totminionskilled'])
df_stats2.head()

print('shape:', df_stats2.shape)

,id,win,kills,deaths,assists,visionscore,totminionskilled
0,1028382,0,7,5,1,0,131
1,1028383,0,0,0,0,0,0
2,1028384,0,0,0,0,0,0
3,1028385,0,0,0,0,0,11
4,1028386,0,0,0,0,0,11


shape: (834518, 7)


### Análise - Estatísticas dos Times

* `matchid` (inteiro): referencia tabela de partidas
* `teamid` (inteiro): referencia times
* `firstblood` (inteiro): time pegou o primeiro abatimento da partida? (ganha bônus de ouro)
* `firsttower` (inteiro): o time destruiu a primeira torre da partida? (ganha bônus de ouro)
* `towerkills` (inteiro): número de torres destruídas
* `inhibkills` (inteiro): número de inibidores destruídos
* `baronkills` (inteiro): número de barões feitos
* `dragonkills` (inteiro): número de dragões feitos 
* `harrykills` (inteiro): arauto foi feito nessa partida?

In [10]:
# df_teamstats = pd.read_csv('./data/teamstats.csv', usecols=['matchid', 'teamid', 'firstblood', 'firsttower', 'towerkills',
#                                                             'inhibkills', 'baronkills', 'dragonkills', 'harrykills'])
df_teamstats = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/teamstats.csv',
                       usecols=['matchid', 'teamid', 'firstblood', 'firsttower', 'towerkills', 'inhibkills', 
                                'baronkills', 'dragonkills', 'harrykills'])
df_teamstats.head()

print('shape:', df_teamstats.shape)

,matchid,teamid,firstblood,firsttower,towerkills,inhibkills,baronkills,dragonkills,harrykills
0,10,100,0,1,5,0,0,0,0
1,10,200,1,0,10,3,1,3,1
2,11,100,1,0,2,0,0,0,0
3,11,200,0,1,10,3,0,2,0
4,12,100,1,0,1,0,0,0,0


shape: (368138, 9)


## Limpeza dos Dados

Para cada uma das mini-bases vamos renomear colunas, concatenar bases, ordenar os dados, conferir os valores únicos de dados categóricos ou o máximo e mínimo de cada coluna de dados numéricos, substituir valores NaN (se for necessário) e, obviamente, limpar todos os dados incorretos ou incompletos.

### Renomeando colunas

O nome `id` é muito vago e pode causar confusões, por isso vamos trocar todos por nomes que façam mais sentido em cada tabela:

In [11]:
df_champs.rename(columns={'id': 'championid'}, inplace=True)
print('df_champs:', list(df_champs))

df_matches.rename(columns={'id': 'matchid'}, inplace=True)
print('df_matches:', list(df_matches))

df_players.rename(columns={'id': 'playerid'}, inplace=True)
print('df_players:', list(df_players))

df_stats1.rename(columns={'id': 'playerid'}, inplace=True)
print('df_stats1:', list(df_stats1))

df_stats2.rename(columns={'id': 'playerid'}, inplace=True)
print('df_stats2:', list(df_stats2))

df_champs: ['name', 'championid']
df_matches: ['matchid', 'queueid', 'seasonid']
df_players: ['playerid', 'matchid', 'player', 'championid', 'role', 'position']
df_stats1: ['playerid', 'win', 'kills', 'deaths', 'assists', 'visionscore', 'totminionskilled']
df_stats2: ['playerid', 'win', 'kills', 'deaths', 'assists', 'visionscore', 'totminionskilled']


### Concatenando bases iguais

Em seguida, vamos concatenar as bases `df_stats1` e `df_stats2`, que possuem os mesmos dados, só foram separadas por serem muito grandes:

In [12]:
df_playerstats = pd.concat([df_stats1, df_stats2])
df_playerstats.head()
df_playerstats.tail()

print('shape:', df_playerstats.shape)

,playerid,win,kills,deaths,assists,visionscore,totminionskilled
0,9,0,6,10,1,14,42
1,10,0,0,2,12,30,17
2,11,0,7,8,5,26,205
3,12,0,5,11,2,5,164
4,13,0,2,8,2,15,235


,playerid,win,kills,deaths,assists,visionscore,totminionskilled
834513,1865600,0,11,10,2,0,292
834514,1865601,0,11,9,7,0,189
834515,1865602,0,4,8,10,0,59
834516,1865603,0,2,11,3,0,53
834517,1865604,0,7,11,9,0,235


shape: (1834517, 7)


Desejamos também juntar as tabelas do jogadores e de suas estatísticas, mas devemos notar o seguinte fator, o número de linhas na tabela dos jogadores é maior que o de suas estatísticas, isto é, existem jogadores sem informações que nós devemos remover:

In [13]:
print('jogadores antes:', df_players.shape[0])
print('informações:', df_playerstats.shape[0])

lista = list(set(df_players['playerid']) - set(df_playerstats['playerid']))
print('jogadores sem informação:', lista)

df_players = df_players[~df_players.playerid.isin(lista)]
print('jogadores agora:', df_players.shape[0])  

print('concatenando...')
df_players = df_players.join(df_playerstats.set_index('playerid'), on='playerid')

df_players.head()

print('shape:', df_players.shape)

jogadores antes: 1834520
informações: 1834517
jogadores sem informação: [1272124, 1272013, 1272175]
jogadores agora: 1834517
concatenando...


,playerid,matchid,player,championid,role,position,win,kills,deaths,assists,visionscore,totminionskilled
0,9,10,1,19,NONE,JUNGLE,0,6,10,1,14,42
1,10,10,2,267,DUO_SUPPORT,BOT,0,0,2,12,30,17
2,11,10,3,119,DUO_CARRY,BOT,0,7,8,5,26,205
3,12,10,4,114,SOLO,TOP,0,5,11,2,5,164
4,13,10,5,112,SOLO,MID,0,2,8,2,15,235


shape: (1834517, 12)


### Trocando "vitória" de tabela

Com as informações dos jogadores todas unificadas na tabela acima, percebemos uma coisa intrigante: por que a informação de vitória está na tabela dos jogadores e não dos times? Em minha visão, é informação desnecessária sendo repetida na tabela errada, então iremos passá-la para a outra tabela. 

Começaremos separando a informação de vitória e o id das partidas e, em seguida, iremos juntar com a tabela `teamstats`:

In [14]:
df = pd.DataFrame({'matchid': df_players.matchid, 'win': df_players.win})

print('valores extraídos da df_players:')
df.head()

df.drop_duplicates(inplace=True)
print('sem linhas duplicadas:')
df.head()


valores extraídos da df_players:


,matchid,win
0,10,0
1,10,0
2,10,0
3,10,0
4,10,0


sem linhas duplicadas:


,matchid,win
0,10,0
5,10,1
10,11,0
15,11,1
20,12,0


Com a informação de vitória separada junto com o id, devemos criar um id único nas duas tabelas a serem unidas (`df` e `df_teamstats`) e concatenar as tabelas baseadas nesse id, supondo que o primeiro time sempre é o número 100 e o segundo é sempre o 200.

Notamos que o número de times é maior que o número de vitórias, então vamos excluir as partidas que não possuem informação de quem ganhou (ou ela está incompleta):

In [15]:
d = df['matchid'].value_counts()
lista = [x for x,y in zip(d.keys().tolist(), d.tolist())  if y < 2]
print('partidas com estatísticas incompletas:', lista)

print('vitórias antes:', df.shape[0])
df = df[~df.matchid.isin(lista)]
print('vitórias agora:', df.shape[0])

print('df_teamstats antes:', df_teamstats.shape[0])
df_teamstats = df_teamstats[~df_teamstats.matchid.isin(lista)]
print('df_teamstats agora:', df_teamstats.shape[0])

print('df_players agora:', df_players.shape[0])
df_players = df_players[~df_players.matchid.isin(lista)]
print('df_players agora:', df_players.shape[0])

print('df_matches agora:', df_matches.shape[0])
df_matches = df_matches[~df_matches.matchid.isin(lista)]
print('df_matches agora:', df_matches.shape[0])

partidas com estatísticas incompletas: [127721, 127704]
vitórias antes: 368136
vitórias agora: 368134
df_teamstats antes: 368138
df_teamstats agora: 368134
df_players agora: 1834517
df_players agora: 1834515
df_matches agora: 184069
df_matches agora: 184067


In [16]:
df['index'] = range(0, len(df))
df_teamstats['index'] = range(0, len(df_teamstats))

df_teamstats = df_teamstats.merge(df.set_index('index'), on=['index', 'matchid'], how='left')
df_teamstats.head()

,matchid,teamid,firstblood,firsttower,towerkills,inhibkills,baronkills,dragonkills,harrykills,index,win
0,10,100,0,1,5,0,0,0,0,0,0.0
1,10,200,1,0,10,3,1,3,1,1,1.0
2,11,100,1,0,2,0,0,0,0,2,0.0
3,11,200,0,1,10,3,0,2,0,3,1.0
4,12,100,1,0,1,0,0,0,0,4,0.0


Agora precisamos limpar as tabelas, da seguinte forma:
* remover `win` da tabela `df_players` e
* remover `index` da tabela `df_teamstats`
* reorganizar as colunas para melhorar legibilidade

In [17]:
del df_players['win']
del df_teamstats['index']

print('sem as colunas:')
df_players.head()
df_teamstats.head()

sem as colunas:


,playerid,matchid,player,championid,role,position,kills,deaths,assists,visionscore,totminionskilled
0,9,10,1,19,NONE,JUNGLE,6,10,1,14,42
1,10,10,2,267,DUO_SUPPORT,BOT,0,2,12,30,17
2,11,10,3,119,DUO_CARRY,BOT,7,8,5,26,205
3,12,10,4,114,SOLO,TOP,5,11,2,5,164
4,13,10,5,112,SOLO,MID,2,8,2,15,235


,matchid,teamid,firstblood,firsttower,towerkills,inhibkills,baronkills,dragonkills,harrykills,win
0,10,100,0,1,5,0,0,0,0,0.0
1,10,200,1,0,10,3,1,3,1,1.0
2,11,100,1,0,2,0,0,0,0,0.0
3,11,200,0,1,10,3,0,2,0,1.0
4,12,100,1,0,1,0,0,0,0,0.0


In [18]:
df_teamstats = df_teamstats.reindex(['matchid', 'teamid', 'win', 'firstblood', 'firsttower', 'towerkills', 'inhibkills', 
                                     'baronkills', 'dragonkills', 'harrykills'], axis=1)
df_teamstats.head()

,matchid,teamid,win,firstblood,firsttower,towerkills,inhibkills,baronkills,dragonkills,harrykills
0,10,100,0.0,0,1,5,0,0,0,0
1,10,200,1.0,1,0,10,3,1,3,1
2,11,100,0.0,1,0,2,0,0,0,0
3,11,200,1.0,0,1,10,3,0,2,0
4,12,100,0.0,1,0,1,0,0,0,0


### Conferindo valores

Gostaríamos de ver, para cada tabela, quais são os valores únicos discretos, quais são os limites para cada atributo, se existem valores errados/NaN a serem removidos ou substituídos e se existem valores duplicados. Para isso, vamos criando pequenos dataframes para mostrar as informações básicas interessantes sobre os pedaços da base:

In [19]:
df = pd.DataFrame({'df_champs': ['championid', 'name'], 
                   'valores duplicados?': [any(df_champs['championid'].duplicated()), 
                                           any(df_champs['name'].duplicated())],
                   'valores NaN?': [df_champs['championid'].isnull().values.any(), 
                                    df_champs['name'].isnull().values.any()],
                   'mesmo tipo?': [all(isinstance(x, int) for x in df_champs.championid),
                                   all(isinstance(x, str) for x in df_champs.name)]
                  })
df

,df_champs,valores duplicados?,valores NaN?,mesmo tipo?
0,championid,False,False,True
1,name,False,False,True


In [20]:
df = pd.DataFrame({'df_matches': ['matchid', 'queueid', 'seasonid'], 
                   'valores duplicados?': [any(df_matches['matchid'].duplicated()), '-', '-'],
                   'valores NaN?': [df_matches['matchid'].isnull().values.any(), 
                                    df_matches['queueid'].isnull().values.any(), 
                                    df_matches['seasonid'].isnull().values.any()],
                   'valores únicos': ['-', df_matches.queueid.unique(), 
                                      df_matches.seasonid.unique()],
                   'mesmo tipo?': [all(isinstance(x, int) for x in df_matches.matchid),
                                   all(isinstance(x, int) for x in df_matches.queueid),
                                   all(isinstance(x, int) for x in df_matches.seasonid),]
                  })

df

,df_matches,valores duplicados?,valores NaN?,valores únicos,mesmo tipo?
0,matchid,False,False,-,True
1,queueid,-,False,"[420, 440, 410, 4, 42, 9, 41]",True
2,seasonid,-,False,"[8, 7, 6, 5, 4, 3]",True


In [21]:
df = pd.DataFrame({'df_players': ['playerid', 'matchid', 'player', 'championid', 'role', 'position', 'kills', 'deaths', 
                                  'assists', 'visionscore', 'totminionskilled'], 
                   'valores duplicados?': [any(df_players['playerid'].duplicated()), '-', '-', '-', '-', '-', '-', '-', 
                                           '-', '-', '-'],
                   'valores NaN?': [df_players['playerid'].isnull().values.any(), 
                                    df_players['matchid'].isnull().values.any(), 
                                    df_players['player'].isnull().values.any(), 
                                    df_players['championid'].isnull().values.any(),
                                    df_players['role'].isnull().values.any(), 
                                    df_players['position'].isnull().values.any(),
                                    df_players['kills'].isnull().values.any(), 
                                    df_players['deaths'].isnull().values.any(),
                                    df_players['assists'].isnull().values.any(),
                                    df_players['visionscore'].isnull().values.any(),
                                    df_players['totminionskilled'].isnull().values.any()],
                   'valores únicos': ['-', '-', df_players.player.unique(), '-', df_players.role.unique(), 
                                      df_players.position.unique(), '-', '-', '-', '-', '-'],
                   'mínimo': ['-', '-', '-', '-', '-', '-', min(df_players.kills), min(df_players.deaths), 
                              min(df_players.assists), min(df_players.visionscore), min(df_players.totminionskilled),],
                   'máximo': ['-', '-', '-', '-', '-', '-', max(df_players.kills), max(df_players.deaths), 
                              max(df_players.assists), max(df_players.visionscore), max(df_players.totminionskilled)],
                   'mesmo tipo?': [all(isinstance(x, int) for x in df_players.playerid),
                                   all(isinstance(x, int) for x in df_players.matchid),
                                   all(isinstance(x, int) for x in df_players.player),
                                   all(isinstance(x, int) for x in df_players.championid),
                                   all(isinstance(x, str) for x in df_players.role),
                                   all(isinstance(x, str) for x in df_players.position),
                                   all(isinstance(x, int) for x in df_players.kills),
                                   all(isinstance(x, int) for x in df_players.deaths),
                                   all(isinstance(x, int) for x in df_players.assists),
                                   all(isinstance(x, int) for x in df_players.visionscore),
                                   all(isinstance(x, int) for x in df_players.totminionskilled)]
                  })

df

,df_players,valores duplicados?,valores NaN?,valores únicos,mínimo,máximo,mesmo tipo?
0,playerid,False,False,-,-,-,True
1,matchid,-,False,-,-,-,True
2,player,-,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",-,-,True
3,championid,-,False,-,-,-,True
4,role,-,False,"[NONE, DUO_SUPPORT, DUO_CARRY, SOLO, DUO]",-,-,True
5,position,-,False,"[JUNGLE, BOT, TOP, MID]",-,-,True
6,kills,-,False,-,0,45,True
7,deaths,-,False,-,0,38,True
8,assists,-,False,-,0,57,True
9,visionscore,-,False,-,0,179,True


In [22]:
df = pd.DataFrame({'df_teamstats': ['matchid', 'teamid', 'win', 'firstblood', 'firsttower', 'towerkills', 'inhibkills', 
                                    'baronkills', 'dragonkills', 'harrykills'], 
                   'valores NaN?': [df_teamstats['matchid'].isnull().values.any(), 
                                    df_teamstats['teamid'].isnull().values.any(),
                                    df_teamstats['win'].isnull().values.any(),
                                    df_teamstats['firstblood'].isnull().values.any(),
                                    df_teamstats['firsttower'].isnull().values.any(), 
                                    df_teamstats['towerkills'].isnull().values.any(),
                                    df_teamstats['inhibkills'].isnull().values.any(), 
                                    df_teamstats['baronkills'].isnull().values.any(),
                                    df_teamstats['dragonkills'].isnull().values.any(),
                                    df_teamstats['harrykills'].isnull().values.any()],
                   'valores únicos': ['-', '-', df_teamstats.win.unique(), df_teamstats.firstblood.unique(), 
                                      df_teamstats.firsttower.unique(), '-', '-', '-', '-', 
                                      df_teamstats.harrykills.unique()],
                   'mínimo': ['-', '-', '-', '-', '-', min(df_teamstats.towerkills), min(df_teamstats.inhibkills), 
                              min(df_teamstats.baronkills), min(df_teamstats.dragonkills), '-'],
                   'máximo': ['-', '-', '-', '-', '-', max(df_teamstats.towerkills), max(df_teamstats.inhibkills), 
                              max(df_teamstats.baronkills), max(df_teamstats.dragonkills), '-'],
                   'mesmo tipo?': [all(isinstance(x, int) for x in df_teamstats.matchid),
                                   all(isinstance(x, int) for x in df_teamstats.teamid),
                                   all(isinstance(x, int) for x in df_teamstats.win),
                                   all(isinstance(x, int) for x in df_teamstats.firstblood),
                                   all(isinstance(x, int) for x in df_teamstats.firsttower),
                                   all(isinstance(x, int) for x in df_teamstats.towerkills),
                                   all(isinstance(x, int) for x in df_teamstats.inhibkills),
                                   all(isinstance(x, int) for x in df_teamstats.baronkills),
                                   all(isinstance(x, int) for x in df_teamstats.dragonkills),
                                   all(isinstance(x, int) for x in df_teamstats.harrykills)]
                  })

df

,df_teamstats,valores NaN?,valores únicos,mínimo,máximo,mesmo tipo?
0,matchid,False,-,-,-,True
1,teamid,False,-,-,-,True
2,win,True,"[0.0, 1.0, nan]",-,-,False
3,firstblood,False,"[0, 1]",-,-,True
4,firsttower,False,"[1, 0]",-,-,True
5,towerkills,False,-,0,16,True
6,inhibkills,False,-,0,13,True
7,baronkills,False,-,0,5,True
8,dragonkills,False,-,0,7,True
9,harrykills,False,"[0, 1, 2]",-,-,True


### Última Limpeza

Até aqui nós percebemos pequenos problemas que tentaremos excluir agora, sendo considerados inconsistências:
1. Os valores na coluna `queueid` na tabela `df_matches` estão, em sua maioria, desatualizados:
    * todos os valores 4 foram substituídos por 420 e todos os valores 9 foram substituídos por 470 na API oficial da Riot Games
    * os valores 41, 42 e 410 foram depreciados na API e serão excluídos da base
    * os valores 470 se referem a outro mapa que não será explorado nesse relatório então eles serão excluídos (consequentemente vamos excluir os valores 9 também)
    * serão então mantidos os valores: 4, 420 e 440

**Obs:** O motivo para a exclusão desses valores é que são estilos diferentes de jogo e mapa, que podem influenciar nas estatísticas aqui obtidas (por exemplo, o mapa Twisted Treeline, que vamos excluir, é 3x3, enquanto o mapa oficial é 5x5; o `queueid` 42 se referia a um estilo de jogo de time fechado, diferente do modo solo/duo, onde você não escolhe a maior parte dos seus companheiros de equipe)

2. Existe um valor na coluna `role` da tabela `df_players` que não condiz com o esperado: o valor `DUO` não informa se a pessoa jogou de `carry` ou `support` e por isso não pode ser considerado nas estatísticas
3. Também iremos remover as linhas que contém absurdos como mais de 1 arauto, mais de 11 torres e mais de 3 inibidores
4. Existem valores NaN na coluna `win` e os demais valores são _float_ sem necessidade
5. Vamos trocar `NONE` porque não é um nome explicativo

1. Removendo valores incorretos no `queueid`:

In [23]:
lista = [x for x,y in zip(df_matches.matchid, df_matches.queueid) if y in [9, 41, 42, 410]]

print('partidas antes:', df_matches.shape[0])
df_matches = df_matches[~df_matches.matchid.isin(lista)]
print('partidas agora:', df_matches.shape[0], '\n')

print('teamstats antes:', df_teamstats.shape[0])
df_teamstats = df_teamstats[~df_teamstats.matchid.isin(lista)]
print('teamstats agora:', df_teamstats.shape[0], '\n')

print('jogadores antes:', df_players.shape[0])
df_players = df_players[~df_players.matchid.isin(lista)]
print('jogadores agora:', df_players.shape[0], '\n')

print('valores únicos da queueid:', df_matches.queueid.unique())

partidas antes: 184067
partidas agora: 177981 

teamstats antes: 368134
teamstats agora: 355962 

jogadores antes: 1834515
jogadores agora: 1779807 

valores únicos da queueid: [420 440   4]


2. Removendo valores incorretos no `role`:

In [24]:
lista = [x for x,y in zip(df_players.playerid, df_players.role) if y == 'DUO']

print('jogadores antes:', df_players.shape[0])
df_players = df_players[~df_players.playerid.isin(lista)]
print('jogadores agora:', df_players.shape[0], '\n')

print('valores únicos da role:', df_players.role.unique())

jogadores antes: 1779807
jogadores agora: 1721459 

valores únicos da role: ['NONE' 'DUO_SUPPORT' 'DUO_CARRY' 'SOLO']


3. Removendo valores incorretos no `towerkills`, no `inhibkills` e no `harrykills`:

In [25]:
lista = [w for w,x,y,z in zip(df_teamstats.matchid, df_teamstats.towerkills, df_teamstats.inhibkills, 
                              df_teamstats.harrykills) if x > 11 or y > 3 or z > 1]

print('partidas antes:', df_matches.shape[0])
df_matches = df_matches[~df_matches.matchid.isin(lista)]
print('partidas agora:', df_matches.shape[0], '\n')

print('teamstats antes:', df_teamstats.shape[0])
df_teamstats = df_teamstats[~df_teamstats.matchid.isin(lista)]
print('teamstats agora:', df_teamstats.shape[0], '\n')

print('jogadores antes:', df_players.shape[0])
df_players = df_players[~df_players.matchid.isin(lista)]
print('jogadores agora:', df_players.shape[0], '\n')

print('mínimo e máximo de towerkills', min(df_teamstats.towerkills), max(df_teamstats.towerkills))
print('mínimo e máximo de inhibkills', min(df_teamstats.inhibkills), max(df_teamstats.inhibkills))
print('valores únicos de harrykills:', df_teamstats.harrykills.unique())

partidas antes: 177981
partidas agora: 163180 

teamstats antes: 355962
teamstats agora: 326360 

jogadores antes: 1721459
jogadores agora: 1576547 

mínimo e máximo de towerkills 0 11
mínimo e máximo de inhibkills 0 3
valores únicos de harrykills: [0 1]


4. Removendo NaN no `win` e passar tipo para inteiro:

In [26]:
index = df_teamstats.loc[df_teamstats['win'].isnull()].index
lista = df_teamstats.iloc[index]['matchid'].values

df_teamstats = df_teamstats.dropna()

print('partidas antes:', df_matches.shape[0])
df_matches = df_matches[~df_matches.matchid.isin(lista)]
print('partidas agora:', df_matches.shape[0], '\n')

print('teamstats antes:', df_teamstats.shape[0])
df_teamstats = df_teamstats[~df_teamstats.matchid.isin(lista)]
print('teamstats agora:', df_teamstats.shape[0], '\n')

print('jogadores antes:', df_players.shape[0])
df_players = df_players[~df_players.matchid.isin(lista)]
print('jogadores agora:', df_players.shape[0], '\n')

df_teamstats['win'] = df_teamstats['win'].astype(int)
print('valores únicos de win:', df_teamstats.win.unique())

partidas antes: 163180
partidas agora: 163167 

teamstats antes: 326347
teamstats agora: 326321 

jogadores antes: 1576547
jogadores agora: 1576417 

valores únicos de win: [0 1]


5. Trocar `NONE` por `JUNGLER`: 

In [27]:
df_players['role'].replace(['NONE'], 'JUNGLER', inplace=True)
df_players.head()

,playerid,matchid,player,championid,role,position,kills,deaths,assists,visionscore,totminionskilled
0,9,10,1,19,JUNGLER,JUNGLE,6,10,1,14,42
1,10,10,2,267,DUO_SUPPORT,BOT,0,2,12,30,17
2,11,10,3,119,DUO_CARRY,BOT,7,8,5,26,205
3,12,10,4,114,SOLO,TOP,5,11,2,5,164
4,13,10,5,112,SOLO,MID,2,8,2,15,235


## Análise Exploratória

A ideia aqui é: ver a distribuição de alguns atributos, suas estatísticas básicas e dar algumas explicações para seus valores; visualizar uma matriz de correlação entre todos os atributos e entre todos os atributos vs vitória; reduzir a dimensionalidade nas bases de jogadores e times.

### Explorando Valores dos Jogadores

Vamos começar averiguando os atributos mais interessantes dos jogadores: `role`, `kills`, `deaths`, `assists`, `visionscore`, `totminionskilled`.

In [28]:
keys_role = df_players['role'].value_counts().keys().tolist()
counts_role = df_players['role'].value_counts().tolist()
total_role = sum(counts_role)
rates_role = [x/total_role for x in counts_role]

keys_kills = df_players['kills'].value_counts().keys().tolist()
counts_kills = df_players['kills'].value_counts().tolist()
total_kills = sum(counts_kills)
rates_kills = [x/total_kills for x in counts_kills]

keys_deaths = df_players['deaths'].value_counts().keys().tolist()
counts_deaths = df_players['deaths'].value_counts().tolist()
total_deaths = sum(counts_deaths)
rates_deaths = [x/total_deaths for x in counts_deaths]

keys_assists = df_players['assists'].value_counts().keys().tolist()
counts_assists = df_players['assists'].value_counts().tolist()
total_assists = sum(counts_assists)
rates_assists = [x/total_assists for x in counts_assists]

trace1 = go.Bar(x=keys_role, y=counts_role, name='role', text=['porcentagem: ' + str(round(x,4)) for x in rates_role])
trace2 = go.Bar(x=keys_kills, y=counts_kills, name='kills', text=['porcentagem: ' + str(round(x,4)) for x in rates_kills])
trace3 = go.Bar(x=keys_deaths, y=counts_deaths, name='deaths', text=['porcentagem: ' + str(round(x,4)) 
                                                                     for x in rates_deaths])
trace4 = go.Bar(x=keys_assists, y=counts_assists, name='assists', text=['porcentagem: ' + str(round(x,4)) 
                                                                        for x in rates_assists])

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('Subplot 1: Role', 'Subplot 2: Kills', 'Subplot 3: Deaths', 
                                                          'Subplot 4: Assists'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(title='Gráficos de distribuição')

plotly.offline.iplot(figure_or_data=fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



Vamos começar vendo a distribuição dos valores de `role`. Na teoria, cada partida possui 5 funções sempre. Como na base as funções de _top laner_ e _mid laner_ são representadas como `SOLO` nós iremos considerar como a mesma função em posições diferentes. Assim, cada função dentre _jungler_, _carry_ e _support_ deveria corresponder à `1/5 (20%)` da base, e a função _solo_ deveria corresponder à `2/5 (40%)` da base. O que corresponde, aproximadamente, ao gráfico mostrado no subplot 1.

Vamos agora conferir algumas estatísticas básicas:

In [29]:
print('kills:')
print('mínimo e máximo:', min(df_players.kills), max(df_players.kills))
print('média:', np.mean(df_players.kills))
print('mediana:', np.median(df_players.kills))
print('variância:', np.var(df_players.kills))
print('desvio-padrão:', np.std(df_players.kills))

kills:
mínimo e máximo: 0 44
média: 5.744466089873428
mediana: 5.0
variância: 20.410269481990824
desvio-padrão: 4.517772623980851


In [30]:
print('deaths:')
print('mínimo e máximo:', min(df_players.deaths), max(df_players.deaths))
print('média:', np.mean(df_players.deaths))
print('mediana:', np.median(df_players.deaths))
print('variância:', np.var(df_players.deaths))
print('desvio-padrão:', np.std(df_players.deaths))

deaths:
mínimo e máximo: 0 38
média: 5.726463873454803
mediana: 6.0
variância: 9.961873227386882
desvio-padrão: 3.15624353106456


In [31]:
print('assists:')
print('mínimo e máximo:', min(df_players.assists), max(df_players.assists))
print('média:', np.mean(df_players.assists))
print('mediana:', np.median(df_players.assists))
print('variância:', np.var(df_players.assists))
print('desvio-padrão:', np.std(df_players.assists))

assists:
mínimo e máximo: 0 52
média: 8.153088300874705
mediana: 7.0
variância: 33.152296662189805
desvio-padrão: 5.7578031107523815


Falando um pouco sobre os três conjuntos de estatísticas para `kills`, `deaths` e `assists`: a primeira coisa que notamos é que os valores são bastante próximos, provavelmente porque são vários jogadores na mesma partida e as estatísticas de um influenciam nas estatísticas de outros. De cara percebemos que o número médio de assists é maior que de _kills_ e _deaths_ e isso pode ser explicado pela facilidade em se conseguir uma assistência: basta causar algum dano no campeão inimigo enquanto o mesmo estiver em luta e for morto nela.

Notamos também que a variância em `kills` e em `assists` é bem maior que em `deaths`: creio que alguns jogadores preferem jogar solo (diminuindo o número de assistências) enquanto outros preferem juntar nas lutas e/ou não consiguem dar o último _hit_ em campeões inimigos ((diminuindo o número de abates).

In [32]:
keys_visionscore = df_players['visionscore'].value_counts().keys().tolist()
counts_visionscore = df_players['visionscore'].value_counts().tolist()

keys_totminionskilled = df_players['totminionskilled'].value_counts().keys().tolist()
counts_totminionskilled = df_players['totminionskilled'].value_counts().tolist()

trace1 = go.Bar(x=keys_visionscore, y=counts_visionscore)
trace2 = go.Bar(x=keys_totminionskilled, y=counts_totminionskilled)

fig = tools.make_subplots(rows=2, cols=1, subplot_titles=('Visionscore', 'Totminionskilled'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)

fig['layout'].update(title='Gráficos de distribuição')

plotly.offline.iplot(figure_or_data=fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [33]:
print('total de pontos:', len(df_players.visionscore))
print('mínimo e máximo:', min(df_players.visionscore), max(df_players.visionscore))
print('média:', np.mean(df_players.visionscore))
print('mediana:', np.median(df_players.visionscore))
print('variância:', np.var(df_players.visionscore))
print('desvio-padrão:', np.std(df_players.visionscore))

total de pontos: 1576417
mínimo e máximo: 0 175
média: 14.078259115449782
mediana: 10.0
variância: 290.7411822163572
desvio-padrão: 17.05113433811244


In [34]:
print('total de pontos:', len(df_players.totminionskilled))
print('mínimo e máximo:', min(df_players.totminionskilled), max(df_players.totminionskilled))
print('média:', np.mean(df_players.totminionskilled))
print('mediana:', np.median(df_players.totminionskilled))
print('variância:', np.var(df_players.totminionskilled))
print('desvio-padrão:', np.std(df_players.totminionskilled))

total de pontos: 1576417
mínimo e máximo: 0 591
média: 119.89593616409871
mediana: 126.0
variância: 6727.67504698034
desvio-padrão: 82.02240576196445


Análisando agora os valores de `visionscore` e `totminionkilleds`. Aqui a diferença entre valores dentro dos atributos é gritante e pode ser explicada pelas características pessoais dos jogadores e de suas funções exercidas (já que não são atributos influenciados por fatores externos), além do nível de habilidade. 

No caso do placar de visão, infelizmente em ranques mais baixos os jogadores não constumam colocar sentinelas de visão no mapa (apesar de visão ser um dos objetivos mais importantes do jogo) e a função de colocar sentinelas fica encabida aos suportes, por isso temos um pico no valor 0. 

Para o total de _farm_ do jogador, precisamos levar em consideração que diferentes funções possuem estilo de _farm_ diferente. Suportes geralmente farmam pouco, em torno de 50 por partida. As outras funções deveriam abater em torno de 200 tropas por partida em tempo normal (20~40 minutos de jogo), mas não é o que geralmente acontece, como podemos notar no gráfico. Em ranques mais baixos é comum as pessoas não saberem farmar e existe a concepção de que matar inimigos é mais importante que conseguir dinheiro e experiência, o que é ridiculamente errôneo (chega um ponto que os campeões podem valer menos que tropas e objetivos). 

É curioso notar que o maior _farm_ é 591. Algumas estratégias fortes que existiram nos metas (mudança em alguma característica do jogo que fortalece ou enfraquecem determinado aspecto do jogo) englobados na base que priorizam o _farm_, geralmente em conjunto com levar torres/objetivos rapidamente. A título de curiosidade, algumas das estratégias mais famosas são: funilamento de recursos (Karthus e Nunu, Master Yi e Taric, dentre outras combinações especializadas em andar sempre em dupla e concentrar _farm_ e ouro em um único campeão), _proxy Singed_ (Singed mata os minions entre as torres na rota do topo, fazendo com que sua tropa puxe mais rápido as torres e evitando conflitos com campeão inimigo), _tower rush_ (foco exclusivamente em tropas e torres, geralmente utilizando Tryndamere, Yorick e Heimerdinger).

### Explorando Valores dos Times

Vamos começar averiguando os atributos mais interessantes dos times: `towerkills`, `inhibkills`, `baronkills`, `dragonkills`, `harrykills`:  

In [ ]:
keys_role = df_players['role'].value_counts().keys().tolist()
counts_role = df_players['role'].value_counts().tolist()
total_role = sum(counts_role)
rates_role = [x/total_role for x in counts_role]

keys_kills = df_players['kills'].value_counts().keys().tolist()
counts_kills = df_players['kills'].value_counts().tolist()
total_kills = sum(counts_kills)
rates_kills = [x/total_kills for x in counts_kills]

keys_deaths = df_players['deaths'].value_counts().keys().tolist()
counts_deaths = df_players['deaths'].value_counts().tolist()
total_deaths = sum(counts_deaths)
rates_deaths = [x/total_deaths for x in counts_deaths]

keys_assists = df_players['assists'].value_counts().keys().tolist()
counts_assists = df_players['assists'].value_counts().tolist()
total_assists = sum(counts_assists)
rates_assists = [x/total_assists for x in counts_assists]

trace1 = go.Bar(x=keys_role, y=counts_role, name='role', text=['porcentagem: ' + str(round(x,4)) for x in rates_role])
trace2 = go.Bar(x=keys_kills, y=counts_kills, name='kills', text=['porcentagem: ' + str(round(x,4)) for x in rates_kills])
trace3 = go.Bar(x=keys_deaths, y=counts_deaths, name='deaths', text=['porcentagem: ' + str(round(x,4)) 
                                                                     for x in rates_deaths])
trace4 = go.Bar(x=keys_assists, y=counts_assists, name='assists', text=['porcentagem: ' + str(round(x,4)) 
                                                                        for x in rates_assists])

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('Subplot 1: Role', 'Subplot 2: Kills', 'Subplot 3: Deaths', 
                                                          'Subplot 4: Assists'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(title='Gráficos de distribuição')

plotly.offline.iplot(figure_or_data=fig)

## Perguntas

Examinando `championid` gostaríamos de ver quais são os campeões mais escolhidos pelos jogadores:

In [ ]:
keys = df_players['championid'].value_counts().keys().tolist()
counts = df_players['championid'].value_counts().tolist()
total = sum(counts)

names = [df_champs.loc[df_champs['championid'] == x]['name'].iloc[0] for x in keys]
rates = [x/total*100 for x in counts]

data = [go.Bar(x=names, y=rates)]

plotly.offline.iplot({"data": data})

## Lixo

**Ideias:**
* visualização dos dados
* cálculo de estatísticas
    * cálculo do kda médio por função (juntar todos os tipos de support)
    * estatística de trinkets colocadas por champion sendo utilizado. Aí cê pensa que quer dividir isso por season --> conferir porcentagem de vitória baseado no placar de visão médio (somatório?)
    * correlação vision score x vitória
    * histograma pra ver a distribuição dos valores do placar de visão (histograma de outros valores também)
* matriz de correlação
* redução de dimensionalidade

data = [go.Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])]

layout = go.Layout(title="hello world")
plotly.offline.iplot({"data": data, "layout": layout})